# Analisador De Correlação Entre Ações

In [16]:
import pandas as pd

from src.correlation import correlation

In [17]:
WALLET = ["PETR4", "AGRO3"]

STOCKS_UNDER_EVALUATION = [
    "ITUB4", "PETZ3", "AZUL4", 
    "CMIG4", "CSNA3", "GGBR4", 
    "GOAU4", "HYPE3", "JBSS3", 
    "KLBN4",  "CMIG4", "CPLE6",
    "LREN3", "SANB11", "BBAS3",
    "CSNA3", "VALE3", "WEGE3",
]

In [18]:
df_results = correlation.calculate_for_wallet(WALLET, STOCKS_UNDER_EVALUATION)

retornando arquivo local:  csv_files/petr4_historical_data.csv
retornando arquivo local:  csv_files/itub4_historical_data.csv
retornando arquivo local:  csv_files/petr4_historical_data.csv
retornando arquivo local:  csv_files/petz3_historical_data.csv
retornando arquivo local:  csv_files/petr4_historical_data.csv
retornando arquivo local:  csv_files/azul4_historical_data.csv
retornando arquivo local:  csv_files/petr4_historical_data.csv
retornando arquivo local:  csv_files/cmig4_historical_data.csv
retornando arquivo local:  csv_files/petr4_historical_data.csv
retornando arquivo local:  csv_files/csna3_historical_data.csv
retornando arquivo local:  csv_files/petr4_historical_data.csv
retornando arquivo local:  csv_files/ggbr4_historical_data.csv
retornando arquivo local:  csv_files/petr4_historical_data.csv
retornando arquivo local:  csv_files/goau4_historical_data.csv
retornando arquivo local:  csv_files/petr4_historical_data.csv
retornando arquivo local:  csv_files/hype3_historical_d

### For correlação Negativa

In [19]:
filter = df_results['Result'].isin(['Forte correlação negativa'])
df_results[filter]

,Stock_Wallet,Stock_Eval,Correlation,Result


### Fraca correlação negativa

In [20]:
filter = df_results['Result'].isin(['Fraca correlação negativa'])
df_results[filter]

,Stock_Wallet,Stock_Eval,Correlation,Result
8,PETR4,JBSS3,-0.39,Fraca correlação negativa
17,PETR4,WEGE3,-0.36,Fraca correlação negativa


### Não correlacionado

In [21]:
filter = df_results['Result'].isin(['Não correlacionado'])
df_results[filter]

,Stock_Wallet,Stock_Eval,Correlation,Result
0,PETR4,ITUB4,-0.18,Não correlacionado
1,PETR4,PETZ3,-0.16,Não correlacionado
2,PETR4,AZUL4,0.22,Não correlacionado
3,PETR4,CMIG4,-0.29,Não correlacionado
5,PETR4,GGBR4,0.08,Não correlacionado
6,PETR4,GOAU4,0.07,Não correlacionado
7,PETR4,HYPE3,0.11,Não correlacionado
9,PETR4,KLBN4,-0.07,Não correlacionado
10,PETR4,CMIG4,-0.29,Não correlacionado
11,PETR4,CPLE6,-0.03,Não correlacionado


### Fraca correlação positiva

In [22]:
filter = df_results['Result'].isin(['Fraca correlação positiva'])
df_results[filter]

,Stock_Wallet,Stock_Eval,Correlation,Result
4,PETR4,CSNA3,0.35,Fraca correlação positiva
14,PETR4,BBAS3,0.40,Fraca correlação positiva
15,PETR4,CSNA3,0.35,Fraca correlação positiva
31,AGRO3,SANB11,0.45,Fraca correlação positiva


### Forte correlação positiva

In [23]:
filter = df_results['Result'].isin(['Forte correlação positiva'])
df_results[filter]

,Stock_Wallet,Stock_Eval,Correlation,Result
